In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from node import Node

## Overview
I want to model the dynamics of a microgrid. Particularly interested in allowing asynchronous updating, meaning that we need an OOP model, rather than a matrix-based one (which would require full-grid timestepping). The trouble with this is that it makes this process much harder to parallelize

Resources:
- NE Real time energy usage dashboard: https://www.eia.gov/beta/electricity/gridmonitor/dashboard/electric_overview/regional/REG-NE

## Try running it!

In [620]:
lat, lon, mu = 42.477642, -71.130318, 0.1  # Boston
get_pos = lambda lat, lon, mu: [np.random.normal(lon, mu), np.random.normal(lat, mu)]

generate_randomly = False
if generate_randomly:
    capacities = np.append(1000, np.zeros(10))
    nodes = np.array([Node(get_pos(lat, lon, mu), storage_capacity=c) for c in capacities])
    
else:
    nodes = np.array([OldHome(get_pos(lat, lon, mu)), OldHome(get_pos(lat, lon, mu)), PowerPlant(get_pos(lat, lon, mu))])
    
grid = EnergyGrid(nodes)
grid.make_neighbors()

# MVP Demo
grid.nodes[0].step(new_demand=10, new_supply=0)
grid.nodes[1].step(new_demand=10, new_supply=0)

In [ ]:
grid.run_simulation()

In [624]:
grid.nodes[0].get_transaction_logs(t_end=pd.to_datetime("2019-07-02 08:00:00"))

,source,dest,amount
timestamp,,,
2019-07-02 05:00:00,<__main__.PowerPlant object at 0x7fce39f99290>,<__main__.OldHome object at 0x7fce9db92390>,0.289172
2019-07-02 06:00:00,<__main__.PowerPlant object at 0x7fce39f99290>,<__main__.OldHome object at 0x7fce9db92390>,0.299073
2019-07-02 07:00:00,<__main__.PowerPlant object at 0x7fce39f99290>,<__main__.OldHome object at 0x7fce9db92390>,0.365433
2019-07-02 08:00:00,<__main__.PowerPlant object at 0x7fce39f99290>,<__main__.OldHome object at 0x7fce9db92390>,0.384823


In [548]:
grid.vis